In [35]:
import numpy as np
import json
from google import Google

class Data():
    def __init__(self, path='../data/Food-2010-7010.json', location_path='../data/location.json'):
        self.path = path
        self.location_path = location_path
        with open(self.path, 'r') as f:
            content = json.loads(f.read(), encoding='utf-8')['articles']
        self.content = [content[i] for i in range(len(content)) if "article_id" in content[i]]
        self.id_list = [self.content[i]["article_id"] for i in range(len(self.content))]
        self.title_list = [self.content[i]["article_title"] for i in range(len(self.content))]
        assert(len(self.id_list) == len(self.title_list) == len(self.content))
        self.num_of_docs = len(self.id_list)  #99745 docs
        
        self.parse_location()
        self.get_stars_and_name()
        self.save_to_json()
    
    def parse_location(self):
        with open(self.location_path, 'r') as f:
            location = json.loads(f.read(), encoding='utf-8') #{"loc_1":[mid_loc of loc_1....]}
        main_location = [key for key, value in location.items()]
        mini_location = [dis for key, value in location.items() for dis in value]
        #### Determin the main & mini location
        count = 0
        for i in range(self.num_of_docs):
            if not self.title_list[i] or '[食記]' not in self.title_list[i]:
                continue
            _title = self.title_list[i].replace("臺","台").strip('[食記]').strip(' ')
            #print(_title)
            tmp_dict = {}
            for j in range(len(_title)-1):
                bigram = ''.join(_title[j:j+2])
                if bigram in main_location:
                    tmp_dict['main_location'] = bigram
                    
                elif bigram in mini_location:
                    if "main_location" in tmp_dict and bigram in location[tmp_dict["main_location"]]:
                        tmp_dict['mini_location'] = bigram
                    else:
                        tmp_dict['mini_location'] = bigram
                        for mini_list in list(location.values()):
                            if bigram in mini_list:
                                tmp_dict['main_location'] = list(location.keys())[list(location.values()).index(mini_list)]
            self.content[i].update(tmp_dict)
    
    def get_stars_and_name(self):
        #self.num_of_docs
        for i in range(10):
            print(i)
            _title = self.title_list[i].replace("臺","台").strip('[食記]').strip(' ').strip('Re:').strip('Fw:')
            tmp_dict = {}
            tmp_obj = Google(query=_title)
            tmp_dict['store_name'] = tmp_obj.name
            tmp_dict['stars'] = tmp_obj.stars
            self.content[i].update(tmp_dict)
    def save_to_json(self):
        outfile = '../data/Food-final.json'
        out_dict = {}
        print('Write {} of docs to output:'.format(len(self.content)))
        out_dict['articles'] = self.content
        with open(outfile, 'w', encoding='utf-8') as f:
            f.write(u'{"articles": [')
        for i in range(self.num_of_docs):
            s = json.dumps(self.content[i], ensure_ascii=False, separators=(',', ':'))
            with open(outfile, 'a', encoding='utf-8') as f:
                f.write(s)
                if i != self.num_of_docs-1:
                    f.write(',\n')
        with open(outfile, 'a', encoding='utf-8') as f:
            f.write(u']}')
        print('Finish writing json file.')
        
data = Data()
for i in range(5):
    print(data.content[i], data.content[i]['store_name'], data.content[i]['stars'])

0
1
2
3
4
5
6
7
8
9
Write 99745 of docs to output:
Finish writing json file.
{'article_title': '[食記]台中崇德路上新開的吃到飽－98 Adagio', 'url': 'https://www.ptt.cc/bbs/Food/M.1389690376.A.D32.html', 'content': '餐廳名稱：98 Adagio 消費時間：2014年/1月(例：2010年/7月，必填項目，違者刪文) 地址：台中市北屯區崇德路2段398號 電話：0422493877 營業時間：08:00A.M. ~ 02:00A.M. 每人平均價位：隨平日例假日與用餐時段而有所不同 有無包廂：N 強烈推薦圖文版: http://zavork2002.blogspot.tw/2014/01/98adagio.html 嘗試一下新首圖，看看有沒有比較切題又有內容。 最近在17Life團購網看到優惠卷，想說不會很遠，吃到飽，食材豐富，早餐時段頗為便 宜(218)，就過來試試。 整體下來還不錯，食物都很好吃，細節部分也都弄得不錯，能選的種類真的很多，就是上 菜太慢了而已，很可惜。 在崇德路二段上，若是從市區過來，經過輕井澤崇德店後沒多久就到了，不難找。 新開幕的氣勢，藉由雪白的外牆外露出來。8點35分到，店外已經停滿一堆摩托車了。 一進門，先看到簡單的櫃檯，似乎單純只是為了收費以及稍微與用餐空間隔開而存在的。 旁邊還放著祝大展鴻圖，生意興隆的好彩頭。 裡面的用餐空間，座位滿多的，但座位之間的間距很小，某些位置行動有點不便。(此為 從裡到外的視角) 雖然是早上8點40分，可是人卻已經這麼多了。看來附近的住戶和上班族消息靈通，即早 卡位。 天花板的裝飾，懸著一個大型盒蓋，外表刻意作成彩色百葉窗造型，讓天頂不至於單調。 中央五點一線地擺置了5台掛燈，希望多少增加一點點豪氣。 自動鋼琴，一直反覆地彈奏著音樂，自動化雖然很有效，但聽起來總是少了這麼一點生氣 。 請原諒不專業的我，根本聽不出來是甚麼曲目，一首也。 收費方式隨平日例假日與用餐時段而有所大大大不同 價格波動起伏很大，從238到1288都有，還要再加10%服務費。 萬一搞錯時段來，而要付與自己所想的不一樣的價格，可就尷尬了。 不願意面對的價位真相結

In [9]:
a = {}
a['今天'] = [1,2]
a['天氣'] = [2,3]
list(a.values()).index([2, 3])
#loca = list(a.keys())[list(a.values()).index(1)]
#loca
#list(a.values()).index(1)

1

In [6]:
from google import Google
test = Google(query='台北 烙張清真牛肉麵館')
test.name, test.stars
test1 = Google(query='台南 北區 連得堂餅家-基本2種+預訂3種')
test1.name, test1.stars

('查看以下內容的搜尋結果：', -5.0)

In [38]:
with open('../data/Food-final.json', 'r') as f:
    content = json.loads(f.read(), encoding='utf-8')['articles']
print(len(content))
print(content[:5])

99745
[{'article_title': '[食記]台中崇德路上新開的吃到飽－98 Adagio', 'url': 'https://www.ptt.cc/bbs/Food/M.1389690376.A.D32.html', 'content': '餐廳名稱：98 Adagio 消費時間：2014年/1月(例：2010年/7月，必填項目，違者刪文) 地址：台中市北屯區崇德路2段398號 電話：0422493877 營業時間：08:00A.M. ~ 02:00A.M. 每人平均價位：隨平日例假日與用餐時段而有所不同 有無包廂：N 強烈推薦圖文版: http://zavork2002.blogspot.tw/2014/01/98adagio.html 嘗試一下新首圖，看看有沒有比較切題又有內容。 最近在17Life團購網看到優惠卷，想說不會很遠，吃到飽，食材豐富，早餐時段頗為便 宜(218)，就過來試試。 整體下來還不錯，食物都很好吃，細節部分也都弄得不錯，能選的種類真的很多，就是上 菜太慢了而已，很可惜。 在崇德路二段上，若是從市區過來，經過輕井澤崇德店後沒多久就到了，不難找。 新開幕的氣勢，藉由雪白的外牆外露出來。8點35分到，店外已經停滿一堆摩托車了。 一進門，先看到簡單的櫃檯，似乎單純只是為了收費以及稍微與用餐空間隔開而存在的。 旁邊還放著祝大展鴻圖，生意興隆的好彩頭。 裡面的用餐空間，座位滿多的，但座位之間的間距很小，某些位置行動有點不便。(此為 從裡到外的視角) 雖然是早上8點40分，可是人卻已經這麼多了。看來附近的住戶和上班族消息靈通，即早 卡位。 天花板的裝飾，懸著一個大型盒蓋，外表刻意作成彩色百葉窗造型，讓天頂不至於單調。 中央五點一線地擺置了5台掛燈，希望多少增加一點點豪氣。 自動鋼琴，一直反覆地彈奏著音樂，自動化雖然很有效，但聽起來總是少了這麼一點生氣 。 請原諒不專業的我，根本聽不出來是甚麼曲目，一首也。 收費方式隨平日例假日與用餐時段而有所大大大不同 價格波動起伏很大，從238到1288都有，還要再加10%服務費。 萬一搞錯時段來，而要付與自己所想的不一樣的價格，可就尷尬了。 不願意面對的價位真相結束了，趕快來介紹食物吧。 事先聲明: 以下這些是平日早上8點早上11點的食材選擇喔，其他時段的我就不清楚了。 相當罕見，早上提供的熱鮮奶和熱